In [8]:
#pip install widgetsnbextension

In [7]:
#pip install ipywidgets

In [17]:
#pip install voila

In [5]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

In [44]:
import ipywidgets as widgets
from IPython.display import display, clear_output, Image
import os

In [5]:
checkpoint = widgets.ToggleButtons(
            options=['lookingglass_dalle_12000.pt','lookingglass_dalle_2000.pt','lookingglass_dalle_4000.pt',
                     'lookingglass_dalle_6000.pt','lookingglass_dalle_8000.pt','lookingglass_dalle_10000.pt',
                     'lookingglass_dalle_12000.pt','lookingglass_dalle_14000.pt','lookingglass_dalle_16000.pt',
                    'lookingglass_dalle_18000.pt','lookingglass_dalle_20000.pt']
        )

In [21]:
confidence = widgets.ToggleButtons(
            options=['Ultra-Low', 'Low', 'Medium', 'High', 'Ultra-High'],description='confidence')
variability = widgets.ToggleButtons(
            options=['Ultra-Low', 'Low', 'Medium', 'High', 'Ultra-High'], description = 'variability')

In [22]:
prompt = widgets.Text(placeholder='Input your prompt')

In [23]:
display(confidence)

ToggleButtons(description='confidence', options=('Ultra-Low', 'Low', 'Medium', 'High', 'Ultra-High'), value='U…

In [24]:
display(variability)

ToggleButtons(description='variability', options=('Ultra-Low', 'Low', 'Medium', 'High', 'Ultra-High'), value='…

In [26]:
display(prompt)

Text(value='', placeholder='Input your prompt')

In [8]:
button_run = widgets.Button(description = 'Run model', tooltip='Send',
                style={'description_width': 'initial'})

In [9]:
output = widgets.Output()

In [10]:
from rudalle import get_rudalle_model, get_tokenizer, get_vae
import torch
from model.functions import generate

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = get_rudalle_model("Malevich", pretrained=True, fp16=True, device=device)

tokenizer --> ready
◼️ Malevich is 1.3 billion params model from the family GPT3-like, that uses Russian language and text+image multi-modality.


In [11]:
vae = get_vae().to("cpu")

Working with z of shape (1, 256, 32, 32) = 262144 dimensions.
vae --> ready


In [45]:
def on_button_clicked(event):
    with output:
        clear_output()
        import os
        model_path = os.path.join(f'checkpoints/{checkpoint.value}')
        filepath = f'{prompt.value}-{checkpoint.value}-{confidence.value}-{variability.value}'
        model.load_state_dict(torch.load(model_path))
        generate(vae, model, prompt.value, confidence = confidence.value, variability = variability.value, rurealesrgan_multiplier="x4", output_filepath=filepath, filter_by_top = 5, image_amount = 9)
        print(f'Images saved in {filepath}')
        
        import os 
        for image in os.listdir(filepath):
            Image(f'filepath/{image}')
            

In [46]:
button_run.on_click(on_button_clicked)

In [47]:
vbox_result = widgets.VBox([button_run, output])

In [48]:
display(button_run, output)

Button(description='Run model', style=ButtonStyle())

Output(outputs=({'output_type': 'error', 'ename': 'UnboundLocalError', 'evalue': "local variable 'os' referenc…